In [1]:
#!/usr/bin/env python3
import matplotlib.animation as animation

import sys
sys.path.append('C:\Program Files\Cubemos\SkeletonTracking\samples\python')
sys.path.append('.\opencv-plot-master')

import matplotlib.pyplot as plt
import os
import numpy as np

from collections import namedtuple
import util as cm
import cv2
import time
import pyrealsense2 as rs
import math
import numpy as np 
from skeletontracker import skeletontracker
import cv2

from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square

import random

%matplotlib qt5

### Helper Functions

In [2]:
# =====================================================
# SKELETON TRACKING FUNCTIONS
# =====================================================

def render_ids_3d(render_image, skeletons_2d, depth_map, depth_intrinsic, joint_confidence):
    
    thickness = 1
    text_color = (255, 255, 255)
    rows, cols, channel = render_image.shape[:3]
    distance_kernel_size = 5
    
    # calculate 3D keypoints and display them
    for skeleton_index in range(len(skeletons_2d)):
        skeleton_2D = skeletons_2d[skeleton_index]
        joints_2D = skeleton_2D.joints
        did_once = False
        for joint_index in range(len(joints_2D)):
            if did_once == False:
                cv2.putText(
                    render_image,
                    "id: " + str(skeleton_2D.id),
                    (int(joints_2D[joint_index].x), int(joints_2D[joint_index].y - 30)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.55,
                    text_color,
                    thickness,
                )
            did_once = True
            # check if the joint was detected and has valid coordinate
            if skeleton_2D.confidences[joint_index] > joint_confidence:
                distance_in_kernel = []
                low_bound_x = max(
                    0,
                    int(
                        joints_2D[joint_index].x - math.floor(distance_kernel_size / 2)
                    ),
                )
                upper_bound_x = min(
                    cols - 1,
                    int(joints_2D[joint_index].x + math.ceil(distance_kernel_size / 2)),
                )
                low_bound_y = max(
                    0,
                    int(
                        joints_2D[joint_index].y - math.floor(distance_kernel_size / 2)
                    ),
                )
                upper_bound_y = min(
                    rows - 1,
                    int(joints_2D[joint_index].y + math.ceil(distance_kernel_size / 2)),
                )
                for x in range(low_bound_x, upper_bound_x):
                    for y in range(low_bound_y, upper_bound_y):
                        distance_in_kernel.append(depth_map.get_distance(x, y))
                median_distance = np.percentile(np.array(distance_in_kernel), 50)
                depth_pixel = [
                    int(joints_2D[joint_index].x),
                    int(joints_2D[joint_index].y),
                ]
                if median_distance >= 0.3:
                    point_3d = rs.rs2_deproject_pixel_to_point(
                        depth_intrinsic, depth_pixel, median_distance
                    )
                    point_3d = np.round([float(i) for i in point_3d], 3)
                    point_str = [str(x) for x in point_3d]
                    
#                     cv2.putText(
#                         render_image,
#                         str(point_3d),
#                         (int(joints_2D[joint_index].x), int(joints_2D[joint_index].y)),
#                         cv2.FONT_HERSHEY_DUPLEX,
#                         0.4,
#                         text_color,
#                         thickness,
#                     )
                    
def compute_orientation(img, display=False):
    
    # apply threshold
    thresh = threshold_otsu(img)
    bw = closing(img > thresh, square(3))

    label_img = label(bw)
    props = regionprops(label_img)

    # largest img
    largest_index = np.argmax([p.area for p in props])
    prop = props[largest_index]
    
    if display: 

        plt.figure()
        plt.imshow(prop.image)

        x0 = prop['Centroid'][1]
        y0 = prop['Centroid'][0]
        x2 = x0 - math.sin(prop['Orientation']) * 0.9 * prop['MinorAxisLength']
        y2 = y0 - math.cos(prop['Orientation']) * 0.9 * prop['MinorAxisLength']

        plt.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
        plt.plot(x0, y0, '.g', markersize=15)

        minr, minc, maxr, maxc = prop['BoundingBox']
        bx = (minc, maxc, maxc, minc, minc)
        by = (minr, minr, maxr, maxr, minr)
        
        plt.plot(bx, by, '-b', linewidth=2.5)
        plt.show()
        
    return prop.orientation

def detect_legs(img, skeleton): 
    
    x_rk, y_rk = int(skeleton.joints[9][0]), int(skeleton.joints[9][1])
    x_ra, y_ra = int(skeleton.joints[10][0]), int(skeleton.joints[10][1])
    x_lk, y_lk = int(skeleton.joints[12][0]), int(skeleton.joints[12][1])
    x_la, y_la = int(skeleton.joints[13][0]), int(skeleton.joints[13][1])
    
    # zero out each leg
    img_left_leg = np.zeros(img.shape,np.uint8)
    img_left_leg[y_lk:y_la, x_la-30:x_la+30] = img[y_lk:y_la, x_la-30:x_la+30]
    img_right_leg = np.zeros(img.shape, np.uint8)
    img_right_leg[y_rk:y_ra, y_ra-30:y_ra+30] = img[y_rk:y_ra, x_ra-30:x_ra+30]

    # threshold each leg 
    gray_left = cv2.cvtColor(img_left_leg, cv2.COLOR_BGR2GRAY)
    ret_left, thresh_left = cv2.threshold(gray_left, 0, 255, cv2.THRESH_OTSU)
    gray_right = cv2.cvtColor(img_right_leg, cv2.COLOR_BGR2GRAY)
    ret_right, thresh_right = cv2.threshold(gray_right, 0, 255, cv2.THRESH_OTSU)

    # find contours for each leg 
    x_l, y_l, w_l, h_l = cv2.boundingRect(thresh_left)
    x_r, y_r, w_r, h_r = cv2.boundingRect(thresh_right)

#     final_img = cv2.rectangle(img, (x_l, y_l), (x_l + w_l, y_l + h_l), (36,255,12), 2)
#     final_img = cv2.rectangle(final_img, (x_r, y_r), (x_r + w_r, y_r + h_r), (36,255,12), 2)

    # display
    #cv2.putText(final_img, 'left leg', (x_l, y_l-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    #cv2.putText(final_img, 'right leg', (x_r, y_r-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imshow('image', final_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def detect_legs_and_angle(img, skeleton, display=False): 
    
    x_rk, y_rk = int(skeleton.joints[9][0]), int(skeleton.joints[9][1])
    x_ra, y_ra = int(skeleton.joints[10][0]), int(skeleton.joints[10][1])
    x_lk, y_lk = int(skeleton.joints[12][0]), int(skeleton.joints[12][1])
    x_la, y_la = int(skeleton.joints[13][0]), int(skeleton.joints[13][1])
        
    # zero out each leg
    img_left_leg = np.zeros(img.shape,np.uint8)
    img_left_leg[y_lk:y_la-15, x_la-70:x_la+70] = img[y_lk:y_la-15, x_la-70:x_la+70]
    img_right_leg = np.zeros(img.shape, np.uint8)
    img_right_leg[y_rk:y_ra-15, x_ra-70:x_ra+70] = img[y_rk:y_ra-15, x_ra-70:x_ra+70]

    # threshold each leg 
    gray_left = cv2.cvtColor(img_left_leg, cv2.COLOR_BGR2GRAY)
    ret_left, thresh_left = cv2.threshold(gray_left, 0, 255, cv2.THRESH_OTSU)
    gray_right = cv2.cvtColor(img_right_leg, cv2.COLOR_BGR2GRAY)
    ret_right, thresh_right = cv2.threshold(gray_right, 0, 255, cv2.THRESH_OTSU)

    # obtain contour
    cntrs_left = cv2.findContours(thresh_left, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs_left = cntrs_left[0] if len(cntrs_left) == 2 else cntrs_left[1]
    cntrs_left = max(cntrs_left, key=cv2.contourArea)
    cntrs_right = cv2.findContours(thresh_right, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntrs_right = cntrs_right[0] if len(cntrs_right) == 2 else cntrs_right[1]
    cntrs_right = max(cntrs_right, key=cv2.contourArea)

    # get bounding box for legs
    x_l, y_l, w_l, h_l = cv2.boundingRect(cntrs_left)
    x_r, y_r, w_r, h_r = cv2.boundingRect(cntrs_right)

    # fit ellipsoid to find orientation 
    ellipse_left, ellipse_right = cv2.fitEllipse(cntrs_left), cv2.fitEllipse(cntrs_right)

    _, angle_l = get_angle_orienation(img, ellipse_left)
    _, angle_r = get_angle_orienation(img, ellipse_right)
    
    print('ANGLES: ', angle_l, angle_r)

    # display 
    if angle_l >= 40 and angle_l <= 120: 
        color_l = (36,255,12)
        status_l = True
    else: 
        status_l = False

    if angle_r >= 50 and angle_r <= 100:
        color_r = (36,255,12)
        status_r = True
    else: 
        status_r = False
        
    cv2.putText(img, str(round(angle_l, 1)), (x_l, y_l-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
    cv2.putText(img, str(round(angle_r, 1)), (x_r, y_r-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
    
    if not status_r or not status_l: 
        cv2.putText(img, 'Leg Status: Bad', (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,12,36), 3)
        
        #cv2.putText(img, str(round(angle_l, 1)), (x_l, y_l-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
        cv2.rectangle(img, (x_l, y_l), (x_l + w_l, y_l + h_l), (255,12,36), 2)

        #cv2.putText(img, str(round(angle_r, 1)), (x_r, y_r-20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,12,36), 2)
        cv2.rectangle(img, (x_r, y_r), (x_r + w_r, y_r + h_r), (255,12,36), 2)
    else: 
        cv2.putText(img, 'Leg Status: Good', (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 3)
    
    if display: 
        cv2.imshow('image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return (status_r and status_l)
        
def get_angle_orienation(img, ellipse, draw_line=False): 
    
    result = None
    (xc, yc), (d1, d2), angle = ellipse
    
    # draw orienation line
    rmajor = max(d1, d2) / 2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    xtop = xc + math.cos(math.radians(angle))*rmajor
    ytop = yc + math.sin(math.radians(angle))*rmajor
    xbot = xc + math.cos(math.radians(angle+180))*rmajor
    ybot = yc + math.sin(math.radians(angle+180))*rmajor
    
    if draw_line:
        result = cv2.line(img, (int(xtop),int(ytop)), (int(xbot),int(ybot)), (0, 0, 255), 3)
    
    return result, angle

def compute_rocking(depth_img, colorized, skeleton, prev_z): 
    
    # extract key points 
    x_ls, y_ls = int(skeleton.joints[5][0]), int(skeleton.joints[5][1])
    x_rs, y_rs = int(skeleton.joints[2][0]), int(skeleton.joints[2][1])
    x_mid, y_mid = int(skeleton.joints[1][0]), int(skeleton.joints[1][1])
    z_ls, z_rs, z_mid = depth_img[y_ls, x_ls], depth_img[y_rs, x_rs], depth_img[y_mid, x_mid]
        
    # compare to last z location 
    avg_position = np.mean([z_ls, z_rs, z_mid])
    
    print('ROCKING: ', prev_z, avg_position)
    
    # display bounding box and status of rocking
    if abs(prev_z - avg_position) > .05: 
        cv2.putText(colorized, 'Rocking Status: Bad', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,12,36), 2)
        cv2.rectangle(colorized, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        return False
    else: 
        cv2.putText(colorized, 'Rocking Status: Good', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
        return True
        
def computed_side_movement(img, skeleton, prev_x): 
    
    # extract key points 
    x_ls, y_ls = int(skeleton.joints[5][0]), int(skeleton.joints[5][1])
    x_rs, y_rs = int(skeleton.joints[2][0]), int(skeleton.joints[2][1])
    
    # compare to previous x value and display 
    avg_position = np.mean([x_ls, x_rs])
    
    print('SIDE MOVEMENT: ', prev_x, avg_position)
    
    if abs(prev_x - avg_position) > 5: 
        cv2.putText(img, 'Side Movement Status: Bad', (20, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,12,36), 2)
        cv2.rectangle(img, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        return False
    else: 
        cv2.putText(img, 'Side Movement Status: Good', (20, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
        return True
    
def compute_shoulder_lifts(img, skeleton, ref_y): 
    
    # extract key points 
    y_rs, y_ls = int(skeleton.joints[2][1]), int(skeleton.joints[5][1])
    avg_pos = np.mean([y_rs, y_ls])
    
    print('SHOULDER LIFT: ', ref_y, avg_pos)
    
    if abs(ref_y - avg_pos) > 5: 
        cv2.putText(img, 'Shoulder Movement Status: Bad', (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,12,36), 2)
        cv2.rectangle(img, (x_rs, y_rs-20), (x_ls + 20, y_ls + 20), (255,12,36), 2)
        return False
    else: 
        cv2.putText(img, 'Shoulder Movement: Good', (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
    
def compute_neck_movement(img, skeleton, ref_neck_pos):
    
    avg_pos = np.mean([int(skeleton.joints[0][0]), int(skeleton.joints[14][0]), int(skeleton.joints[15][0]), int(skeleton.joints[16][0]), int(skeleton.joints[17][0])])
    print('NECK MOVEMENT: ', ref_neck_pos, avg_pos)
    if abs(ref_neck_pos - avg_pos) > 10: 
        cv2.putText(img, 'Neck Movement Status: Bad', (20, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,12,36), 2)
    else: 
        cv2.putText(img, 'Neck Movement Status: Good', (20, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2) 
    
def convert_depth_frame_to_pointcloud(depth_image, camera_intrinsics ):
    """
    Convert the depthmap to a 3D point cloud
    Parameters:
    -----------
    depth_frame : rs.frame() The depth_frame containing the depth map
    camera_intrinsics : The intrinsic values of the imager in whose coordinate system the depth_frame is computed
    Return:
    ----------
    x : array
        The x values of the pointcloud in meters
    y : array
        The y values of the pointcloud in meters
    z : array
        The z values of the pointcloud in meters
    """

    [height, width] = depth_image.shape

    nx = np.linspace(0, width-1, width)
    ny = np.linspace(0, height-1, height)
    u, v = np.meshgrid(nx, ny)
    x = (u.flatten() - camera_intrinsics.ppx)/camera_intrinsics.fx
    y = (v.flatten() - camera_intrinsics.ppy)/camera_intrinsics.fy

    z = depth_image.flatten() / 1000;
    x = np.multiply(x,z)
    y = np.multiply(y,z)

    x = x[np.nonzero(z)]
    y = y[np.nonzero(z)]
    z = z[np.nonzero(z)]

    return x, y, z
               
# =====================================================
# CHEST VOLUME FUNCTIONS
# =====================================================

### Realtime Skeleton Tracking & Positioning

In [3]:
#
# Configuration and Variable Setup
#

# configure depth and color streams of the intel realsense
config = rs.config()
#config.enable_device('013422062082')
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# start the realsense pipeline
pipeline = rs.pipeline()
cfg = pipeline.start(config)

# get intrinsics
profile = cfg.get_stream(rs.stream.depth) # Fetch stream profile for depth stream
camera_instrinsics = profile.as_video_stream_profile().get_intrinsics()

# create necessary realsense objects
colorizer = rs.colorizer()
align = rs.align(rs.stream.color)
filters = [rs.decimation_filter(),
           rs.disparity_transform(True),
           rs.spatial_filter(),
           rs.temporal_filter(), 
           rs.disparity_transform(False),
           rs.hole_filling_filter()]

# initialize the cubemos api (skeleton tracking)
skeletrack = skeletontracker(cloud_tracking_api_key="")
joint_confidence = 0.2

# create window for initialisation
window_name = "cubemos skeleton tracking with realsense D400 series"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL + cv2.WINDOW_KEEPRATIO)

# initialize variables
skeleton_tracking_results = [] 
chest_depth_rois = []
chest_color_rois = []
colorized_depths = []
chest_displacements = []
pixel_to_meters_scaling =  0.0010000000474974513

#
# Realtime Skeleton Tracking
#

i = 0 
while True:
    
    #
    # Collecting Necessary Data From RealSense
    #
    
    # Create a pipeline object. 
    # This object configures the streaming camera and owns it's handle
    unaligned_frames = pipeline.wait_for_frames()
    
    # obtain alligned depth and color images
    frames = align.process(unaligned_frames)
    depth = frames.get_depth_frame()
    color = frames.get_color_frame()
    
    # skip if not any are missing 
    if not depth or not color:
        continue
    
    # post process filtering for depth image 
    for f in filters:
        depth = f.process(depth)
    depth = depth.as_depth_frame()   
    depth_intrinsic = depth.profile.as_video_stream_profile().intrinsics
    
    # Convert frames to necessary numpy arrays 
    depth_image = np.asanyarray(depth.get_data())
    depth_image_color = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.1), cv2.COLORMAP_JET)
    depth_scaled = depth_image * pixel_to_meters_scaling

    color_image = np.asanyarray(color.get_data())
    color_image = cv2.resize(color_image, dsize=(depth_image.shape[1], depth_image.shape[0]), interpolation=cv2.INTER_CUBIC)
    color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
    
    #
    # Run Skeleton Tracking & Movement Detections
    #
    
    # perform inference and update the tracking id
    skeletons = skeletrack.track_skeletons(color_image)
    skeleton_tracking_results.append(skeletons)
    
    # render leg position on image 
    if len(skeletons) > 0: 
        try: 
            leg_position_status = detect_legs_and_angle(depth_image_color, skeletons[0])
        except: 
            None
        try:    
            if i == 0: 
                original_x = np.mean([skeletons[0].joints[5][0], skeletons[0].joints[2][0]])
            side_movement_status = computed_side_movement(depth_image_color, skeletons[0], original_x)
        except Exception as e: 
            print('error in side to side movement')
            print(e)
        try: 
            if i == 0: 
                x_ls, y_ls = int(skeletons[0].joints[5][0]), int(skeletons[0].joints[5][1])
                x_rs, y_rs = int(skeletons[0].joints[2][0]), int(skeletons[0].joints[2][1])
                x_mid, y_mid = int(skeletons[0].joints[1][0]), int(skeletons[0].joints[1][1])
                z_ls, z_rs, z_mid = depth_scaled[y_ls, x_ls], depth_scaled[y_rs, x_rs], depth_scaled[y_mid, x_mid]
                ref_z = np.mean([z_ls, z_rs, z_mid])
            rocking_status = compute_rocking(depth_scaled, depth_image_color, skeletons[0], ref_z)
        except Exception as e: 
            print('error in rocking')
            print(e)
        try:
            if i == 0: 
                ref_shoulder_height = np.mean([y_rs, y_ls])
            shoulder_lift_status = compute_shoulder_lifts(depth_image_color, skeletons[0], ref_shoulder_height)
        except Exception as e: 
            print('error shoulder height')
            print(e)
        try:
            if i == 0: 
                neck_features = [int(skeletons[0].joints[0][0]), int(skeletons[0].joints[14][0]), int(skeletons[0].joints[15][0]), int(skeletons[0].joints[16][0]), int(skeletons[0].joints[17][0])]
                ref_neck = np.mean(neck_features)
            compute_neck_movement(depth_image_color, skeletons[0], ref_neck)
        except Exception as e: 
            print('error neck movement')
            print(e)
    i += 1
    
    cm.render_result(skeletons, depth_image_color, joint_confidence)
    render_ids_3d(depth_image_color, skeletons, depth, depth_intrinsic, joint_confidence)
    
    #
    # Obtain Chest Region of Interests & 
    # Compute chest displacement and volume 
    #
        
    # obtain region of interest for chest using skeleton points 
    curr_y_ls, curr_y_rs = int(skeletons[0].joints[5][1]), int(skeletons[0].joints[2][1])
    curr_x_la, curr_x_ra = int(skeletons[0].joints[6][0]), int(skeletons[0].joints[3][0])
    curr_x_lw, curr_y_lw = int(skeletons[0].joints[11][0]), int(skeletons[0].joints[11][1])
    curr_x_rw, curr_y_rw = int(skeletons[0].joints[8][0]), int(skeletons[0].joints[8][1])
    
    roi_y0 = max(curr_y_ls, curr_y_rs) # top height of roi 
    roi_y1 = min(curr_y_lw, curr_y_rw) # bottom height of roi 
    roi_y1 = int(roi_y0 + ((roi_y1 - roi_y0) * .7)) # bottom height of roi 
        
    chest_depth_rois.append(depth_scaled[roi_y0:roi_y1,curr_x_ra:curr_x_la])
    chest_color_rois.append(depth_image_color[roi_y0:roi_y1,curr_x_ra:curr_x_la])
    colorized_depths.append(depth_image_color)
    chest_displacements.append(np.mean(depth_scaled[roi_y0:roi_y1,curr_x_ra:curr_x_la]))

    cv2.imshow(window_name, depth_image_color)
    if cv2.waitKey(33) == 27:
        break

pipeline.stop()
cv2.destroyAllWindows()
plt.show()

Initialising the Skeleton Tracking Pipeline with EDGE tracking.
SIDE MOVEMENT:  504.96705627441406 504.5
ROCKING:  1.8836667561360325 1.8836667561360325
SHOULDER LIFT:  13.5 13.5
NECK MOVEMENT:  201.8 201.8
SIDE MOVEMENT:  504.96705627441406 284.0
ROCKING:  1.8836667561360325 0.6150000292109326
SHOULDER LIFT:  13.5 95.0
NECK MOVEMENT:  201.8 198.0


C:\Users\hengf\anaconda3\envs\fheng_env\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\hengf\anaconda3\envs\fheng_env\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SIDE MOVEMENT:  504.96705627441406 504.5
ROCKING:  1.8836667561360325 1.888000089675188
SHOULDER LIFT:  13.5 12.0
NECK MOVEMENT:  201.8 201.8
SIDE MOVEMENT:  504.96705627441406 504.5
ROCKING:  1.8836667561360325 1.886333422929359
SHOULDER LIFT:  13.5 12.0
NECK MOVEMENT:  201.8 201.8
SIDE MOVEMENT:  504.96705627441406 504.5
ROCKING:  1.8836667561360325 1.8816667560410376
SHOULDER LIFT:  13.5 12.0
NECK MOVEMENT:  201.8 201.8
SIDE MOVEMENT:  504.96705627441406 506.0
ROCKING:  1.8836667561360325 1.882333422739369
SHOULDER LIFT:  13.5 12.0
NECK MOVEMENT:  201.8 201.8
SIDE MOVEMENT:  504.96705627441406 504.5
ROCKING:  1.8836667561360325 1.8873334229768564
SHOULDER LIFT:  13.5 12.0
NECK MOVEMENT:  201.8 201.8
SIDE MOVEMENT:  504.96705627441406 506.0
ROCKING:  1.8836667561360325 1.8896667564210172
SHOULDER LIFT:  13.5 12.0
NECK MOVEMENT:  201.8 201.2
SIDE MOVEMENT:  504.96705627441406 504.5
ROCKING:  1.8836667561360325 1.8866667562785249
SHOULDER LIFT:  13.5 13.5
NECK MOVEMENT:  201.8 201.2
SI

### Visualize Results

In [129]:
#color_temp = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.1), cv2.COLORMAP_JET)

plt.figure()
plt.imshow(depth_image_color)

for i, coord in enumerate(skeleton_tracking_results[10][0].joints): 
        plt.text(coord[0], coord[1], str(i), fontsize=9, color='white')

In [65]:
skeleton_tracking_results[20]

[SkeletonKeypoints(joints=[Coordinate(x=367.1208801269531, y=105.46875), Coordinate(x=344.6153869628906, y=147.65625), Coordinate(x=313.67034912109375, y=150.46875), Coordinate(x=299.6044006347656, y=203.90625), Coordinate(x=330.5494689941406, y=232.03125), Coordinate(x=378.3736267089844, y=147.65625), Coordinate(x=392.4395751953125, y=198.28125), Coordinate(x=369.93408203125, y=226.40625), Coordinate(x=322.1098937988281, y=229.21875), Coordinate(x=288.3516540527344, y=260.15625), Coordinate(x=308.0439758300781, y=358.59375), Coordinate(x=367.1208801269531, y=229.21875), Coordinate(x=403.69232177734375, y=260.15625), Coordinate(x=389.6263732910156, y=358.59375), Coordinate(x=361.4945068359375, y=97.03125), Coordinate(x=367.1208801269531, y=97.03125), Coordinate(x=338.989013671875, y=105.46875), Coordinate(x=-1.0, y=-1.0)], confidences=[0.9598100781440735, 0.8198184967041016, 0.8097206950187683, 0.8367859721183777, 0.8338071703910828, 0.8672670125961304, 0.8345627784729004, 0.8307352066

In [66]:
skeleton_tracking_results[21]

[SkeletonKeypoints(joints=[Coordinate(x=367.1208801269531, y=105.46875), Coordinate(x=344.6153869628906, y=147.65625), Coordinate(x=313.67034912109375, y=147.65625), Coordinate(x=299.6044006347656, y=203.90625), Coordinate(x=330.5494689941406, y=232.03125), Coordinate(x=378.3736267089844, y=147.65625), Coordinate(x=392.4395751953125, y=198.28125), Coordinate(x=372.74725341796875, y=226.40625), Coordinate(x=322.1098937988281, y=229.21875), Coordinate(x=288.3516540527344, y=260.15625), Coordinate(x=308.0439758300781, y=358.59375), Coordinate(x=367.1208801269531, y=229.21875), Coordinate(x=403.69232177734375, y=262.96875), Coordinate(x=389.6263732910156, y=358.59375), Coordinate(x=361.4945068359375, y=97.03125), Coordinate(x=367.1208801269531, y=97.03125), Coordinate(x=338.989013671875, y=105.46875), Coordinate(x=-1.0, y=-1.0)], confidences=[0.96529221534729, 0.8248725533485413, 0.8111159801483154, 0.8490180969238281, 0.8487850427627563, 0.8680945038795471, 0.8288558721542358, 0.846559882